<a href="https://colab.research.google.com/github/MiguelUrenaPliego/GeoimageVision/blob/main/examples/example_irregularity_norms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Irregularity module example

This module computes footprint irregularity indices of buildings

Install libraries if using colab

In [1]:
!pip install geopandas
!pip install git+https://github.com/GeomaticsCaminosUPM/footprint_attributes.git
!pip install folium matplotlib mapclassify

  Cloning https://github.com/GeomaticsCaminosUPM/footprint_attributes.git to /tmp/pip-req-build-zmytrhrw
  Running command git clone --filter=blob:none --quiet https://github.com/GeomaticsCaminosUPM/footprint_attributes.git /tmp/pip-req-build-zmytrhrw
  Resolved https://github.com/GeomaticsCaminosUPM/footprint_attributes.git to commit 73248fcbb964b9a726602eaf06a0567b4dddc3b8
  Preparing metadata (setup.py) ... done
  Created wheel for SeismicBuildingExposure: filename=SeismicBuildingExposure-0.1.0-py3-none-any.whl size=17367 sha256=e3f0447cad8fad1055e3c2b796661fe792fe2d22c9d539829e95d26d944b6919
  Stored in directory: /tmp/pip-ephem-wheel-cache-3itv7j6c/wheels/7f/bd/c1/7a442bce743433d970ec52e18518fa8d82cb93ef44a7babcbf
Successfully built SeismicBuildingExposure
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 1.6 MB/s eta 0:00:00


In [2]:
import footprint_attributes.irregularity
import geopandas as gpd

Documentation of every function can be found using `help()` or under https://github.com/GeomaticsCaminosUPM/SeismicBuildingExposure

In [3]:
help(footprint_attributes.irregularity.eurocode_8_df)

Help on function eurocode_8_df in module footprint_attributes.irregularity:

eurocode_8_df(geoms: geopandas.geodataframe.GeoDataFrame) -> pandas.core.frame.DataFrame



## 1. Load data

Load the footprints geodataframe in `.shp` `.gpkg` `.geojson` format using `geopandas.read_file()`

Download an example footprints file if needed

In [4]:
!curl -L -o footprints.gpkg https://github.com/GeomaticsCaminosUPM/footprint_attributes/raw/refs/heads/main/examples/footprints.gpkg

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  364k  100  364k    0     0   685k      0 --:--:-- --:--:-- --:--:-- 2044k


Load the footprints geodataframe in `.shp` `.gpkg` `.geojson` format using `geopandas.read_file()`

In [5]:
footprints_gdf = gpd.read_file('footprints.gpkg')

- All geometries should be single part and polygons.

    - Multi part geometries (MultiPolygon) would mean two buildings are contained in the same footprint geometry which should not happen.

    - Footprints must be polygons, not linestrings.

Check if all geometries are `Polygon`

In [6]:
if any(footprints_gdf.geometry.type == 'MultiPolygon'):
    print("There are multiplart geometries.")

if any(footprints_gdf.geometry.type.str.contains('Polygon') == False):
    print("Some geometries are not Polygon")

There are multiplart geometries.


Explode multipart geometries into single parts if needed


Note: The row number will be reset. Maybe you do not know how to match the footprints with your data. An id column is a good idea to solve this issue.

In [7]:
footprints_gdf = footprints_gdf.explode().reset_index() # The new index column is the row number in the origninal dataframe
footprints_gdf

,index,gid,geometry
0,0,1218.0,"POLYGON ((488600.239 1097577.417, 488591.045 1..."
1,1,1219.0,"POLYGON ((488562.593 1097585.096, 488567.403 1..."
2,2,1220.0,"POLYGON ((488601.331 1097581.438, 488599.351 1..."
3,3,1221.0,"POLYGON ((488517.642 1097593.605, 488517.592 1..."
4,4,1222.0,"POLYGON ((488492.697 1097591.081, 488492.546 1..."
...,...,...,...
939,943,1273.0,"POLYGON ((488958.217 1097631.443, 488961.413 1..."
940,944,1333.0,"POLYGON ((489119.978 1097677.125, 489130.663 1..."
941,945,3632.0,"POLYGON ((489024.946 1097763.628, 489024.157 1..."
942,946,1505.0,"POLYGON ((488924.894 1097824.766, 488928.927 1..."


## 2. Irregularity norms



### 2.1. Eurocode 8

In [10]:
eurocode_8 = footprint_attributes.irregularity.eurocode_8_df(footprints_gdf)
eurocode_8

,excentricity_ratio,radius_ratio,slenderness,compactness,angle_excentricity,angle_slenderness
0,0.245076,0.349005,1.232601,1.000000,-82.598668,-75.086654
1,0.008968,0.163555,3.192047,0.999683,-16.032497,-86.104788
2,0.044199,0.279863,2.961852,0.999880,45.056284,-65.468554
3,0.014583,0.232617,3.338191,0.997752,-52.235437,-4.542072
4,0.255400,0.223321,2.101109,0.859100,88.389159,-10.539551
...,...,...,...,...,...,...
939,0.622027,0.214324,1.100164,0.909876,-36.951387,-6.732787
940,0.011061,0.297200,1.158917,1.000000,-56.189202,-12.724535
941,0.185631,0.340129,1.739688,0.875405,45.307141,-76.976761
942,0.158282,0.439820,1.350890,0.893873,-13.396754,-62.699325


In [15]:
footprints_gdf['compactness_EC8'] = eurocode_8['compactness'] # Save desired columns in footprints_gdf

### 2.2. Costa Rica building code

In [18]:
costa_rica = footprint_attributes.irregularity.codigo_sismico_costa_rica_df(footprints_gdf)
costa_rica

,excentricity_ratio,angle
0,0.032608,-81.520791
1,0.001232,-3.748371
2,0.009637,33.439118
3,0.001942,-54.645441
4,0.039722,84.707905
...,...,...
939,0.053691,-36.706984
940,0.001342,-56.123849
941,0.032035,41.327770
942,0.030572,-12.771424


In [19]:
footprints_gdf['excentricity_ratio_CR'] = costa_rica['excentricity_ratio'] # Save desired columns in footprints_gdf

### 2.3. Mexico NTC norm

In [23]:
mexico = footprint_attributes.irregularity.mexico_NTC_df(footprints_gdf)
mexico

,setback_ratio,hole_ratio
0,0.000000,0.0
1,0.000000,0.0
2,0.000000,0.0
3,0.000000,0.0
4,0.691307,0.0
...,...,...
939,0.394207,0.0
940,0.000000,0.0
941,0.520152,0.0
942,0.476095,0.0


In [24]:
footprints_gdf['setback_ratio_NTC'] = mexico['setback_ratio'] # Save desired columns in footprints_gdf

## 3. Example code to plot results


Plot a map (needs pip install folium matplotlib mapclassify)

### 3.1. Compactness Eurocode 8

In [28]:
footprints_gdf.explore(
    column='compactness_EC8',       # Column to base the color on
    cmap='RdYlGn',        # Color map (Yellow to Red)
    legend=True ,           # Add a legend
    vmin=0.8,
    vmax=1
)

### 3.2. Excentricity ratio Costa Rica building code

In [32]:
footprints_gdf.explore(
    column='excentricity_ratio_CR',       # Column to base the color on
    cmap='RdYlGn_r',        # Color map (Yellow to Red)
    legend=True ,           # Add a legend
    vmin=0,
    vmax=0.15
)

### 3.3. Setback ratio Mexico NTC

In [36]:
footprints_gdf.explore(
    column='setback_ratio_NTC',       # Column to base the color on
    cmap='RdYlGn_r',        # Color map (Yellow to Red)
    legend=True ,           # Add a legend
    vmin=0,
    vmax=0.4
)

## 4. Save results

In [37]:
footprints_gdf.to_file('file.gpkg')